In [1]:
# import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('wordnet_ic')

In [2]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from numpy import linalg as LA
import math
from ortools.linear_solver import pywraplp

In [3]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head(2)

(743, 43)


,X.AUTHID,anger_doc_sum,anticipation_doc_sum,arousal_doc_sum,disgust_doc_sum,dominance_doc_sum,fear_doc_sum,joy_doc_sum,neg_doc_sum,num_text_words,...,cSIN_label_fin,cEXC_label_fin,cCOM_label_fin,cRUG_label_fin,cSOP_label_fin,cSIN_fin_rank,cEXC_fin_rank,cCOM_fin_rank,cRUG_fin_rank,cSOP_fin_rank
0,b'http://blog.searsholdings.com/leadership-vie...,1,9,31.317,0,38.293,7,16,7,124,...,n,y,y,n,y,5.0,1.0,2.0,3.0,4.0
1,b'https://atyourservice.blogs.xerox.com/catego...,0,4,23.260,0,32.665,0,1,2,126,...,y,n,y,n,y,NaN,NaN,NaN,NaN,NaN


In [4]:
site_content = data['site.content']
docs = {}
for i in range(site_content.size):
    docs[i] = eval(site_content[i]).decode('utf-8')

In [5]:
original_corpus = [1]*len(docs)
corpus = [1]*len(docs)
copy_corpus = [1]*len(docs)

for i in range(len(docs)):
    original_corpus[i] = sent_tokenize(docs[i])
    corpus[i] = sent_tokenize(docs[i])
    copy_corpus[i] = sent_tokenize(docs[i])

In [6]:
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
remove_punctuation_map = dict((ord(char), ' ') for char in string.punctuation)
integer_chars = [str(digit) for digit in list(range(10))]
remove_integer_map = dict((ord(char), None) for char in integer_chars)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def lemm_tokens(tokens):
    return [lemmatizer.lemmatize(item) for item in tokens]

def filtersent(text, remove_stopwords=True, stem=True, lemm=True, avoid_single_char=True):
    # remove puctuations and lower the case
    simpletext = text.lower().translate(remove_punctuation_map).translate(remove_integer_map)
    # tokenize
    words = nltk.word_tokenize(simpletext)
    # remove stop words
    if remove_stopwords:
        words = [w for w in words if w not in stopwords.words('english')]
    # lemmatize them
    if lemm:
        words = lemm_tokens(words)
    # stem them
    if stem:
        words = stem_tokens(words)
    # avoid single character words
    if avoid_single_char:
        words = [w for w in words if w not in string.ascii_lowercase]
    # detikenise the sentence
    return TreebankWordDetokenizer().detokenize(words)

In [7]:
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        corpus[i][j] = filtersent(original_corpus[i][j])
        copy_corpus[i][j] = original_corpus[i][j]

In [8]:
for i in range(len(corpus)):
    idx = []
    for j in range(len(corpus[i])):
        if(corpus[i][j] == ''):
            idx.insert(0,j)
    for index in idx:
        del corpus[i][index]
        del copy_corpus[i][index]

In [9]:
# count matrix
count_word = [1]*len(corpus)
count_matrix = [1]*len(corpus)
count_persent = [1]*len(corpus) # no of words per sentence

vectorizer = CountVectorizer()
for i in range(len(corpus)):
    count_word[i] = vectorizer.fit_transform(corpus[i])
    count_matrix[i] = count_word[i].toarray()
    count_persent[i] = np.sum(count_matrix[i], axis=1)

In [10]:
def position_score(pos):
    if(pos<=0.1):
        return 0.17
    elif(pos<=0.2):
        return 0.23
    elif(pos<=0.3):
        return 0.14
    elif(pos<=0.4):
        return 0.08
    elif(pos<=0.5):
        return 0.05
    elif(pos<=0.6):
        return 0.04
    elif(pos<=0.7):
        return 0.06
    elif(pos<=0.8):
        return 0.04
    elif(pos<=0.9):
        return 0.04
    else:
        return 0.15

In [11]:
def pos_score(pos_tag):
    return 1

In [12]:
def fam_score(fam):
    if(fam==0):
        return 1
    return 1/(1+np.exp( -8 * (-0.5 + 1/fam) ))

In [13]:
mu = 0.85
lam = 0.7
k = 3

top_corpus = [1]*len(corpus)

for d in range(len(corpus)):
# if(True):
#     d = 1
    
    no_sent = len(corpus[d])
    document = TreebankWordDetokenizer().detokenize([string for string in corpus[d]])
    
    words = nltk.tokenize.word_tokenize(document)
    fdist1 = nltk.FreqDist(words)

#      tf word
    tf_word = {word:freq for word, freq in fdist1.items() if not word.isdigit()}
    
    # length of word
    len_word = {word: len(word) for word in fdist1.keys()}
    
    # pos tag
    pos_word = {word:pos_score(nltk.pos_tag([word])[0][1]) for word in fdist1.keys()}
    
    # familiarity
    fam_word = {word: fam_score(len(wn.synsets(word))) for word in fdist1.keys()}
    
    score_sents = []
    for i in range(no_sent):
        score_sent = 0
        
        for j,word in enumerate(nltk.tokenize.word_tokenize(corpus[d][i])):
            # occurence score
            occ_word = position_score(j/len(corpus[d][i]))
            
            score_word = tf_word[word] * len_word[word] * pos_word[word] * fam_word[word] * occ_word
            score_sent += score_word
            
            # corefferant
            if(pos_word[word]=='PRP'):
                if(j/len(corpus[s][i]) < 0.5):
                    if(i>0):
                        score_sent += score_sents[i-1]/len(nltk.tokenize.word_tokenize(corpus[d][i-1]))
                else:
                    score_sent += score_sent/len(nltk.tokenize.word_tokenize(corpus[d][i]))
        
        score_sents.append(score_sent)
    
    # similarity matrix
    sim = np.zeros((no_sent, no_sent))
    worthless_sent = []
    for x in range(no_sent):
        for y in range(no_sent):
            adotb = np.dot(count_matrix[d][x], count_matrix[d][y])
            a = LA.norm(count_matrix[d][x])
            b = LA.norm(count_matrix[d][y])
            if(x==y):
                sim[x,y] = (1-mu)/no_sent
            else:
                sim[x,y] = mu*(adotb/(a*b)) + (1-mu)/no_sent
    
    denom = np.sum(sim,axis=1)
    for x in range(no_sent):
        sim[x,:] = sim[x,:] * 1/denom[x]
        
    # Markov's process
    v = np.array(score_sents)
    
    while(True):
        v_dash = v.dot(sim)
        if(LA.norm(v_dash - v) < 0.000000001):
            break
        v = v_dash
    
    alpha = {}
    beta = {}
    redundacy = sim;

    # ILP selection
    solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    for i in range(no_sent):
        alpha[i] = solver.BoolVar('alpha[%i]' % i)

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            beta[i,j] = solver.BoolVar('beta[%i,%i]' %(i,j))


    solver.Maximize(solver.Sum(alpha[i]*lam*v[i] for i in range(no_sent)) - solver.Sum([beta[i,j]*(1-lam)*redundacy[i,j] for j in range(i+1,no_sent) for i in range(no_sent)]))

    # constraints
    solver.Add( solver.Sum([alpha[i] for i in range(no_sent)]) <= k )

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            solver.Add( beta[i,j] <= alpha[i])
            solver.Add( beta[i,j] <= alpha[j])
            solver.Add( alpha[i] + alpha[j] <= 1 + beta[i,j])

    sol = solver.Solve()
    
    alpha_sol = [alpha[i].solution_value() for i in range(no_sent)]
    top_sents = []
    for i in range(no_sent):
        if alpha_sol[i]==1:
            top_sents.append(copy_corpus[d][i])
    top_corpus[d] = top_sents
    
    if(d%10 == 0):
        print(d,end=".")

0.10.20.30.40.50.60.70.80.90.100.110.120.130.140.150.160.170.180.190.200.210.220.230.240.250.260.270.280.290.300.310.320.330.340.350.360.370.380.390.400.410.420.430.440.450.460.470.480.490.500.510.520.530.540.550.560.570.580.590.600.610.620.630.640.650.660.670.680.690.700.710.720.730.740.

In [14]:
top_data = pd.DataFrame({"X.AUTHID": data["X.AUTHID"],
                        "FEA1": [ s[0] for s in top_corpus],
                         "FEA2": [ s[1] for s in top_corpus],
                         "FEA3": [ s[2] for s in top_corpus]})
top_data.head(5)

,X.AUTHID,FEA1,FEA2,FEA3
0,b'http://blog.searsholdings.com/leadership-vie...,SHC Speaks by As Chief HR Officer and co-e spo...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...
1,b'https://atyourservice.blogs.xerox.com/catego...,When you look at your Xerox device and see the...,Recently new support videos were posted to the...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...
4,b'http://blog.searsholdings.com/2016/07/page/2/',"For our Danbury Sears store, this meant that w...","On Saturday, July 2, Sears celebrated the gran...",More than 20 Sears associates volunteered with...


In [15]:
top_corpus[20]

['Ill be hosting a unique set of customersat ;, SYN109, (Wednesday May 24, 4:45pm) where well have a discussion and audience-driven Q&A with 4 Citrix Cloud customers:;Well cut the marketing fluff.',
 "Just candid, matter-of-fact discussions about:;Ken is Sr. Director of Product Marketing in Citrix's Cloud Services group, and spends most of his time helping lead the charge to converge concepts of cloud computing, virtual workspaces, and hosted service providers.",
 'One of the key differentiators ; Citrix VP of WW Sales and Strategy, Ash Vijayakanthan shares his thoughts on the benefits Citrix Cloud brings for Citrix Service Providers ; NetScaler Management and Analytics from Citrix allows administrators to easily view, automate, and manage all their application infrastructure, as well ;Dont worry, you can unsubscribe at anytime.']

In [16]:
VAD_data = pd.read_csv("Lexicons/NRC_VAD_Lexicon_new2.csv")
LEX_data = pd.read_csv("Lexicons/NRC_lexicon_new2.csv")

LEX_data.head(2)

,word,emotion,score
0,abacus,trust,1
1,abandon,fear,1


In [17]:
VAD_word = {VAD_data.iloc[i,0]:np.sum(VAD_data.iloc[i,1:]) for i in range(len(VAD_data))}

In [18]:
word_dict = set(LEX_data.iloc[:,0])
LEX_word = {word:0 for word in word_dict}
for word in LEX_data.iloc[:,0]:
    LEX_word[word] += 1

In [19]:
LEX_word["unpaid"]

3

In [20]:
mu = 0.85
lam = 0.7
k = 3
smooth_const = 0.5

top_corpus = [1]*len(corpus)

for d in range(len(corpus)):
# if(True):
#     d = 0
    
    no_sent = len(corpus[d])
    document = TreebankWordDetokenizer().detokenize([string for string in corpus[d]])
    
    words = nltk.tokenize.word_tokenize(document)
    fdist1 = nltk.FreqDist(words)

#      tf word
    tf_word = {word:freq for word, freq in fdist1.items() if not word.isdigit()}
    
    # length of word
    len_word = {word: len(word) for word in fdist1.keys()}
    
    # pos tag
    pos_word = {word:pos_score(nltk.pos_tag([word])[0][1]) for word in fdist1.keys()}
    
    # familiarity
    fam_word = {word: fam_score(len(wn.synsets(word))) for word in fdist1.keys()}
    
    # Lexiscores
    lex_word = {word:smooth_const for word in fdist1.keys()}
    for word in lex_word.keys():
        if(word in LEX_word.keys()):
            lex_word[word] += LEX_word[word]
        if(word in VAD_word.keys()):
            lex_word[word] += VAD_word[word]
    
    score_sents = []
    for i in range(no_sent):
        score_sent = 0
        
        for j,word in enumerate(nltk.tokenize.word_tokenize(corpus[d][i])):
            # occurence score
            occ_word = position_score(j/len(corpus[d][i]))
            
            score_word = tf_word[word] * len_word[word] * pos_word[word] * fam_word[word] * lex_word[word] * occ_word
            score_sent += score_word
            
            # corefferant
            if(pos_word[word]=='PRP'):
                if(j/len(corpus[s][i]) < 0.5):
                    if(i>0):
                        score_sent += score_sents[i-1]/len(nltk.tokenize.word_tokenize(corpus[d][i-1]))
                else:
                    score_sent += score_sent/len(nltk.tokenize.word_tokenize(corpus[d][i]))
        
        score_sents.append(score_sent)
    
    # similarity matrix
    sim = np.zeros((no_sent, no_sent))
    worthless_sent = []
    for x in range(no_sent):
        for y in range(no_sent):
            adotb = np.dot(count_matrix[d][x], count_matrix[d][y])
            a = LA.norm(count_matrix[d][x])
            b = LA.norm(count_matrix[d][y])
            if(x==y):
                sim[x,y] = (1-mu)/no_sent
            else:
                sim[x,y] = mu*(adotb/(a*b)) + (1-mu)/no_sent
    
    denom = np.sum(sim,axis=1)
    for x in range(no_sent):
        sim[x,:] = sim[x,:] * 1/denom[x]
        
    # Markov's process
    v = np.array(score_sents)
    
    while(True):
        v_dash = v.dot(sim)
        if(LA.norm(v_dash - v) < 0.000000001):
            break
        v = v_dash
    
    alpha = {}
    beta = {}
    redundacy = sim;

    # ILP selection
    solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    for i in range(no_sent):
        alpha[i] = solver.BoolVar('alpha[%i]' % i)

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            beta[i,j] = solver.BoolVar('beta[%i,%i]' %(i,j))


    solver.Maximize(solver.Sum(alpha[i]*lam*v[i] for i in range(no_sent)) - solver.Sum([beta[i,j]*(1-lam)*redundacy[i,j] for j in range(i+1,no_sent) for i in range(no_sent)]))

    # constraints
    solver.Add( solver.Sum([alpha[i] for i in range(no_sent)]) <= k )

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            solver.Add( beta[i,j] <= alpha[i])
            solver.Add( beta[i,j] <= alpha[j])
            solver.Add( alpha[i] + alpha[j] <= 1 + beta[i,j])

    sol = solver.Solve()
    
    alpha_sol = [alpha[i].solution_value() for i in range(no_sent)]
    top_sents = []
    for i in range(no_sent):
        if alpha_sol[i]==1:
            top_sents.append(copy_corpus[d][i])
    top_corpus[d] = top_sents
    
    if(d%10 == 0):
        print(d,end=".")

0.10.20.30.40.50.60.70.80.90.100.110.120.130.140.150.160.170.180.190.200.210.220.230.240.250.260.270.280.290.300.310.320.330.340.350.360.370.380.390.400.410.420.430.440.450.460.470.480.490.500.510.520.530.540.550.560.570.580.590.600.610.620.630.640.650.660.670.680.690.700.710.720.730.740.

In [21]:
top_data["FEA_NRC1"] = [s[0] for s in top_corpus]
top_data["FEA_NRC2"] = [s[1] for s in top_corpus]
top_data["FEA_NRC3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,FEA1,FEA2,FEA3,FEA_NRC1,FEA_NRC2,FEA_NRC3
0,b'http://blog.searsholdings.com/leadership-vie...,SHC Speaks by As Chief HR Officer and co-e spo...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...,SHC Speaks by As Chief HR Officer and co-e spo...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...
1,b'https://atyourservice.blogs.xerox.com/catego...,When you look at your Xerox device and see the...,Recently new support videos were posted to the...,Xerox and Xerox and Design are trademarks of X...,When you look at your Xerox device and see the...,Recently new support videos were posted to the...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...
4,b'http://blog.searsholdings.com/2016/07/page/2/',"For our Danbury Sears store, this meant that w...","On Saturday, July 2, Sears celebrated the gran...",More than 20 Sears associates volunteered with...,"For our Danbury Sears store, this meant that w...","On Saturday, July 2, Sears celebrated the gran...",More than 20 Sears associates volunteered with...


In [22]:
top_data.to_csv('topdata_featbase.csv',encoding='utf-8', index=False)